# DEAM Dataset - Feed Forward Neural Network
## Essentia Best Overall & openSMILE GeMAPS Featureset

## Import relevant libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torcheval.metrics import R2Score

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import math

import sys
sys.path.insert(1, '../../utils')
from paths import *

## Define Functions for Neural Networks

Define function for converting dataframes to PyTorch tensors

In [2]:
def string_to_numpy_array(str):
  values = str.strip('[]').split()
  return np.array([float(value)for value in values]).flatten()

def convert_ndarray_col_to_tensor(df_object_col):
  # the df_object_col may be of the type object, but it is actually a numpy.ndarray
  df_object_col = df_object_col.apply(string_to_numpy_array)
  return torch.tensor(df_object_col.tolist(), dtype=torch.float64)

## Neural Network Training

### Import annotations dataset

In [3]:
df_annotations = pd.read_csv(get_deam_path('processed/annotations/deam_static_annotations.csv'))
df_annotations

,song_id,valence_mean_mapped,arousal_mean_mapped
0,2,-0.475,-0.500
1,3,-0.375,-0.425
2,4,0.175,0.125
3,5,-0.150,0.075
4,7,0.200,0.350
...,...,...,...
1739,1996,-0.275,0.225
1740,1997,0.075,-0.275
1741,1998,0.350,0.300
1742,1999,-0.100,0.100


### Import the featureset

In [4]:
df_essentia_best_overall_opensmile_gemaps_features = pd.read_csv(get_deam_path('processed/features/integrated/essentia_best_overall_opensmile_gemaps_features.csv'))

# drop Unnamed:0 column
df_essentia_best_overall_opensmile_gemaps_features = df_essentia_best_overall_opensmile_gemaps_features[df_essentia_best_overall_opensmile_gemaps_features.columns[1:]]

df_essentia_best_overall_opensmile_gemaps_features

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,2,9.415085,14.939523,309.038300,742.725952,328.973969,21.801605,11.508560,-1.316976,37.293823,...,-8.027659,15.973524,-0.024486,-0.007114,2.375139,0.311180,3.170000,3.689382,0.037143,0.026573
1,3,17.002226,26.332752,547.728210,1229.172241,471.216980,76.052628,62.377014,-1.212672,54.853020,...,-21.117159,30.598803,-0.104570,-0.008604,1.865837,1.957295,0.402500,0.548327,0.104521,0.106012
2,4,8.181362,12.390743,164.146927,348.634216,182.409042,16.516722,11.165314,-1.461427,19.692038,...,-9.712925,16.788680,-0.052437,-0.006673,3.131941,1.868327,0.473095,0.416942,0.054833,0.033689
3,5,6.160454,9.577818,82.103508,203.349884,131.002609,14.615296,9.967463,-1.718094,15.931263,...,-7.968155,13.642329,0.054624,-0.023808,2.310084,0.267082,3.697500,3.349172,0.038571,0.033987
4,7,46.677437,67.838478,2727.447998,6007.274902,635.005981,99.851807,80.275414,-1.255423,84.649658,...,-24.957670,34.760834,-0.055579,-0.009088,4.639290,0.377862,2.605294,3.186398,0.041667,0.039756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,1996,6.549002,10.465438,90.671638,229.975418,102.836098,12.534721,10.126063,-1.418290,11.308396,...,-7.664728,11.855576,-0.004562,-0.020547,5.512336,1.023815,0.933913,1.089083,0.032593,0.018377
1740,1997,5.866978,9.743977,73.712753,198.091965,95.177231,11.646033,7.612457,-1.583036,12.044659,...,-8.445706,14.445628,-0.003737,-0.019565,4.578795,0.868403,1.104872,1.144158,0.043500,0.041021
1741,1998,8.790737,14.096998,345.523193,891.726868,402.426819,15.330372,10.513874,-1.687109,19.856863,...,-7.466240,15.266773,-0.035106,-0.014033,4.512114,2.939212,0.269924,0.482651,0.061154,0.043816
1742,1999,7.967627,12.597425,119.048744,285.454956,249.734558,20.406567,17.666672,-0.742111,17.207710,...,-8.933933,15.910338,-0.012904,-0.014340,3.311847,2.448253,0.348182,0.316909,0.051875,0.039784


In [5]:
df_essentia_best_overall_opensmile_gemaps_features.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1744 entries, 0 to 1743
Data columns (total 126 columns):
 #    Column                                          Dtype  
---   ------                                          -----  
 0    song_id                                         int64  
 1    lowlevel.melbands_kurtosis.dmean                float64
 2    lowlevel.melbands_kurtosis.dmean2               float64
 3    lowlevel.melbands_kurtosis.dvar                 float64
 4    lowlevel.melbands_kurtosis.dvar2                float64
 5    lowlevel.melbands_kurtosis.max                  float64
 6    lowlevel.melbands_kurtosis.mean                 float64
 7    lowlevel.melbands_kurtosis.median               float64
 8    lowlevel.melbands_kurtosis.min                  float64
 9    lowlevel.melbands_kurtosis.stdev                float64
 10   lowlevel.melbands_kurtosis.var                  float64
 11   lowlevel.melbands_skewness.dmean                float64
 12   lowlevel.melbands_

Join both the featureset and annotation set together

In [6]:
df_essentia_best_overall_opensmile_gemaps_whole = pd.merge(df_essentia_best_overall_opensmile_gemaps_features, df_annotations, how='inner', on='song_id')
df_essentia_best_overall_opensmile_gemaps_whole = df_essentia_best_overall_opensmile_gemaps_whole.drop('song_id', axis=1)
df_essentia_best_overall_opensmile_gemaps_whole

,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,lowlevel.melbands_kurtosis.var,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,valence_mean_mapped,arousal_mean_mapped
0,9.415085,14.939523,309.038300,742.725952,328.973969,21.801605,11.508560,-1.316976,37.293823,1390.829224,...,-0.024486,-0.007114,2.375139,0.311180,3.170000,3.689382,0.037143,0.026573,-0.475,-0.500
1,17.002226,26.332752,547.728210,1229.172241,471.216980,76.052628,62.377014,-1.212672,54.853020,3008.853760,...,-0.104570,-0.008604,1.865837,1.957295,0.402500,0.548327,0.104521,0.106012,-0.375,-0.425
2,8.181362,12.390743,164.146927,348.634216,182.409042,16.516722,11.165314,-1.461427,19.692038,387.776367,...,-0.052437,-0.006673,3.131941,1.868327,0.473095,0.416942,0.054833,0.033689,0.175,0.125
3,6.160454,9.577818,82.103508,203.349884,131.002609,14.615296,9.967463,-1.718094,15.931263,253.805130,...,0.054624,-0.023808,2.310084,0.267082,3.697500,3.349172,0.038571,0.033987,-0.150,0.075
4,46.677437,67.838478,2727.447998,6007.274902,635.005981,99.851807,80.275414,-1.255423,84.649658,7165.564941,...,-0.055579,-0.009088,4.639290,0.377862,2.605294,3.186398,0.041667,0.039756,0.200,0.350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,6.549002,10.465438,90.671638,229.975418,102.836098,12.534721,10.126063,-1.418290,11.308396,127.879837,...,-0.004562,-0.020547,5.512336,1.023815,0.933913,1.089083,0.032593,0.018377,-0.275,0.225
1740,5.866978,9.743977,73.712753,198.091965,95.177231,11.646033,7.612457,-1.583036,12.044659,145.073792,...,-0.003737,-0.019565,4.578795,0.868403,1.104872,1.144158,0.043500,0.041021,0.075,-0.275
1741,8.790737,14.096998,345.523193,891.726868,402.426819,15.330372,10.513874,-1.687109,19.856863,394.295044,...,-0.035106,-0.014033,4.512114,2.939212,0.269924,0.482651,0.061154,0.043816,0.350,0.300
1742,7.967627,12.597425,119.048744,285.454956,249.734558,20.406567,17.666672,-0.742111,17.207710,296.105286,...,-0.012904,-0.014340,3.311847,2.448253,0.348182,0.316909,0.051875,0.039784,-0.100,0.100


#### Prepare dataframes for the neural network

Perform splitting of the dataframe into training and testing sets

In [7]:
features = df_essentia_best_overall_opensmile_gemaps_features.drop('song_id', axis=1)
features

,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,lowlevel.melbands_kurtosis.var,...,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,9.415085,14.939523,309.038300,742.725952,328.973969,21.801605,11.508560,-1.316976,37.293823,1390.829224,...,-8.027659,15.973524,-0.024486,-0.007114,2.375139,0.311180,3.170000,3.689382,0.037143,0.026573
1,17.002226,26.332752,547.728210,1229.172241,471.216980,76.052628,62.377014,-1.212672,54.853020,3008.853760,...,-21.117159,30.598803,-0.104570,-0.008604,1.865837,1.957295,0.402500,0.548327,0.104521,0.106012
2,8.181362,12.390743,164.146927,348.634216,182.409042,16.516722,11.165314,-1.461427,19.692038,387.776367,...,-9.712925,16.788680,-0.052437,-0.006673,3.131941,1.868327,0.473095,0.416942,0.054833,0.033689
3,6.160454,9.577818,82.103508,203.349884,131.002609,14.615296,9.967463,-1.718094,15.931263,253.805130,...,-7.968155,13.642329,0.054624,-0.023808,2.310084,0.267082,3.697500,3.349172,0.038571,0.033987
4,46.677437,67.838478,2727.447998,6007.274902,635.005981,99.851807,80.275414,-1.255423,84.649658,7165.564941,...,-24.957670,34.760834,-0.055579,-0.009088,4.639290,0.377862,2.605294,3.186398,0.041667,0.039756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,6.549002,10.465438,90.671638,229.975418,102.836098,12.534721,10.126063,-1.418290,11.308396,127.879837,...,-7.664728,11.855576,-0.004562,-0.020547,5.512336,1.023815,0.933913,1.089083,0.032593,0.018377
1740,5.866978,9.743977,73.712753,198.091965,95.177231,11.646033,7.612457,-1.583036,12.044659,145.073792,...,-8.445706,14.445628,-0.003737,-0.019565,4.578795,0.868403,1.104872,1.144158,0.043500,0.041021
1741,8.790737,14.096998,345.523193,891.726868,402.426819,15.330372,10.513874,-1.687109,19.856863,394.295044,...,-7.466240,15.266773,-0.035106,-0.014033,4.512114,2.939212,0.269924,0.482651,0.061154,0.043816
1742,7.967627,12.597425,119.048744,285.454956,249.734558,20.406567,17.666672,-0.742111,17.207710,296.105286,...,-8.933933,15.910338,-0.012904,-0.014340,3.311847,2.448253,0.348182,0.316909,0.051875,0.039784


In [8]:
targets = df_annotations.drop('song_id', axis=1)
targets

,valence_mean_mapped,arousal_mean_mapped
0,-0.475,-0.500
1,-0.375,-0.425
2,0.175,0.125
3,-0.150,0.075
4,0.200,0.350
...,...,...
1739,-0.275,0.225
1740,0.075,-0.275
1741,0.350,0.300
1742,-0.100,0.100


Perform 80-20 train-test split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

Create tensors for X_train and X_test

In [10]:
# get the columns containing numpy.ndarray as its values
numpy_array_columns = X_train.select_dtypes(include=['object']).columns.tolist()

print(f'Numpy ndarray columns: {numpy_array_columns}')

# tensors for columns that are numpy.ndarray
X_train_tensors_ndarray = [convert_ndarray_col_to_tensor(X_train[col]) for col in numpy_array_columns]
X_test_tensors_ndarray = [convert_ndarray_col_to_tensor(X_test[col]) for col in numpy_array_columns]

# generate a tensor for the remaining dataframe (for columns where the type is not numpy.ndarray)
X_train = X_train.drop(numpy_array_columns, axis=1)
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float64)
X_test = X_test.drop(numpy_array_columns, axis=1)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float64)

Numpy ndarray columns: ['rhythm.beats_loudness_band_ratio.dmean', 'rhythm.beats_loudness_band_ratio.dmean2', 'rhythm.beats_loudness_band_ratio.dvar', 'rhythm.beats_loudness_band_ratio.dvar2', 'rhythm.beats_loudness_band_ratio.max', 'rhythm.beats_loudness_band_ratio.mean', 'rhythm.beats_loudness_band_ratio.median', 'rhythm.beats_loudness_band_ratio.min', 'rhythm.beats_loudness_band_ratio.stdev', 'rhythm.beats_loudness_band_ratio.var', 'tonal.chords_histogram']


/var/folders/ft/gzvmkgdj727g157scc1xw9f80000gn/T/ipykernel_36314/291577459.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(df_object_col.tolist(), dtype=torch.float64)


Create tensors for Y_train and Y_test

In [11]:
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float64)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float64)

Define neural network parameters and instantitate neural network

In [12]:
input_size = len(numpy_array_columns) + 1 # total number of tensors
hidden_size = 20 
output_size = 2  # Output size for valence and arousal
learning_rate = 0.001
criterion = nn.MSELoss()
num_epochs = 300

Define the neural network

In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), math.ceil((input_size**0.5) * 2)),
            nn.ReLU(),
            nn.Linear(math.ceil((input_size**0.5) * 2), 2)
        )

    def forward(self, x):
        return self.layers(x)

#### Training

Prepare input_train_data and target_train_labels

In [14]:
X_train_tensors = [X_train_tensor]
X_train_tensors.extend(X_train_tensors_ndarray)

# Find the maximum shape among all tensors
max_shape = max([tensor.shape for tensor in X_train_tensors])

# Adjust tensor shapes to match the maximum shape without data loss
X_train_tensors_adjusted = []

for tensor in X_train_tensors:
    padding = torch.zeros(*max_shape)
    padding_slice = [slice(0, s) for s in tensor.shape]
    padding[tuple(padding_slice)] = tensor
    X_train_tensors_adjusted.append(padding)

print(X_train_tensors_adjusted)

[tensor([[2.3157e+00, 3.6614e+00, 2.0211e+01,  ..., 1.6105e+01, 0.0000e+00,
         0.0000e+00],
        [2.9117e+01, 4.7916e+01, 1.2388e+03,  ..., 8.3958e-01, 7.1111e-02,
         5.1551e-02],
        [1.1072e+01, 1.8110e+01, 8.1454e+02,  ..., 2.7032e-01, 5.6905e-02,
         4.3149e-02],
        ...,
        [2.7215e+00, 4.6408e+00, 5.7143e+00,  ..., 1.3252e+00, 5.4615e-02,
         3.6992e-02],
        [1.0655e+01, 1.5671e+01, 3.8495e+02,  ..., 6.1123e-01, 4.8810e-02,
         4.2325e-02],
        [5.1972e+00, 8.6743e+00, 4.1878e+01,  ..., 3.3433e-01, 6.3854e-02,
         5.1768e-02]]), tensor([[0.2067, 0.2477, 0.2203,  ..., 0.0000, 0.0000, 0.0000],
        [0.0517, 0.0445, 0.0071,  ..., 0.0000, 0.0000, 0.0000],
        [0.2552, 0.1026, 0.1676,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.1426, 0.1382, 0.0952,  ..., 0.0000, 0.0000, 0.0000],
        [0.2343, 0.1386, 0.0801,  ..., 0.0000, 0.0000, 0.0000],
        [0.2475, 0.1573, 0.0822,  ..., 0.0000, 0.0000, 0.0000]]), ten

In [15]:
input_train_data = torch.stack(tuple(X_train_tensors_adjusted), dim=0)

input_train_data = input_train_data.view(input_train_data.shape[1], -1)
print(input_train_data.shape)

target_train_labels = y_train_tensor

torch.Size([1395, 1368])


Training loop

In [16]:
model = NeuralNetwork(input_size=input_train_data.shape[1])
optimiser = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
  optimiser.zero_grad()
  
  # forward pass
  output = model(input_train_data)

  # calculate loss
  loss = torch.sqrt(criterion(output.float(), target_train_labels.float()))

  # backward pass
  loss.backward()
  # update weights
  optimiser.step()

  print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

print("Training completed.")

Epoch 1, Loss: 2.5330731868743896
Epoch 2, Loss: 5.685922145843506
Epoch 3, Loss: 2.7010087966918945
Epoch 4, Loss: 4.889945030212402
Epoch 5, Loss: 2.566559314727783
Epoch 6, Loss: 1.9862488508224487
Epoch 7, Loss: 2.798041343688965
Epoch 8, Loss: 1.649936318397522
Epoch 9, Loss: 0.8810027241706848
Epoch 10, Loss: 0.8255035877227783
Epoch 11, Loss: 0.37942954897880554
Epoch 12, Loss: 0.7643644213676453
Epoch 13, Loss: 0.926055371761322
Epoch 14, Loss: 0.5126396417617798
Epoch 15, Loss: 0.8699385523796082
Epoch 16, Loss: 0.3391343653202057
Epoch 17, Loss: 1.1642701625823975
Epoch 18, Loss: 0.7571074366569519
Epoch 19, Loss: 0.6630001664161682
Epoch 20, Loss: 0.5613374710083008
Epoch 21, Loss: 0.45618176460266113
Epoch 22, Loss: 0.4769607186317444
Epoch 23, Loss: 0.40985944867134094
Epoch 24, Loss: 0.6689966917037964
Epoch 25, Loss: 0.33953267335891724
Epoch 26, Loss: 0.6619608402252197
Epoch 27, Loss: 0.37472110986709595
Epoch 28, Loss: 0.676660418510437
Epoch 29, Loss: 0.5548718571662

#### Testing

Prepare input_test_data and target_test_labels

In [17]:
X_test_tensors = [X_test_tensor]
X_test_tensors.extend(X_test_tensors_ndarray)

# Find the maximum shape among all tensors
max_shape = max([tensor.shape for tensor in X_test_tensors])

# Adjust tensor shapes to match the maximum shape without data loss
X_test_tensors_adjusted = []

for tensor in X_test_tensors:
    padding = torch.zeros(*max_shape)
    padding_slice = [slice(0, s) for s in tensor.shape]
    padding[tuple(padding_slice)] = tensor
    X_test_tensors_adjusted.append(padding)

print(X_test_tensors_adjusted)

[tensor([[9.1150e+00, 1.4027e+01, 1.4427e+02,  ..., 6.0066e-01, 4.6531e-02,
         4.2163e-02],
        [4.4607e+00, 6.6783e+00, 7.0000e+01,  ..., 7.3430e+00, 3.2000e-02,
         1.4697e-02],
        [1.0269e+01, 1.5653e+01, 2.8315e+02,  ..., 2.6254e-01, 6.9222e-02,
         5.6258e-02],
        ...,
        [2.3918e+01, 3.0684e+01, 2.3833e+03,  ..., 2.7756e-01, 4.1875e-02,
         3.2829e-02],
        [5.7581e+00, 9.1960e+00, 1.2349e+02,  ..., 1.3992e+00, 5.7586e-02,
         5.2958e-02],
        [7.2809e+00, 1.1810e+01, 1.2758e+02,  ..., 1.7965e+00, 3.4167e-02,
         2.1969e-02]]), tensor([[0.2487, 0.1433, 0.1589,  ..., 0.0000, 0.0000, 0.0000],
        [0.1954, 0.2819, 0.1756,  ..., 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1484, 0.1311,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0670, 0.0161, 0.0082,  ..., 0.0000, 0.0000, 0.0000],
        [0.3329, 0.1905, 0.0814,  ..., 0.0000, 0.0000, 0.0000],
        [0.1787, 0.1682, 0.0755,  ..., 0.0000, 0.0000, 0.0000]]), ten

In [18]:
input_test_data = torch.stack(tuple(X_test_tensors_adjusted), dim=0)

input_test_data = input_test_data.view(input_test_data.shape[1], -1)
print(input_test_data.shape)

target_test_labels = y_test_tensor

torch.Size([349, 1368])


Generating scores

In [19]:
with torch.no_grad():
  test_pred = model(input_test_data)
  test_loss = criterion(test_pred.float(), target_test_labels)

print(f'Test loss: {test_loss.item()}')

metric = R2Score()
metric.update(test_pred, target_test_labels)
r2_score = metric.compute()
print(f'R2 score: {r2_score.item()}')


Test loss: 0.13183933620280527
R2 score: -0.4339191286118643
